In [107]:
import gensim
import pickle
from nltk.stem import WordNetLemmatizer
import json
import re
from nltk.tokenize import word_tokenize
from nltk.sentiment.util import mark_negation
import nltk
import os
import utils
from gensim.models import Word2Vec,FastText
from sklearn.decomposition import PCA
from matplotlib import pyplot

## Constants

In [108]:
data_dir="data/dataset/"
pretrained_vectors='data/GoogleNews-vectors-negative300.bin.gz'

## Read Input

In [109]:
x=[]
for dir,subdir,files in os.walk(data_dir):
    for file in files:
        file_path=os.path.join(dir,file)
        x+=utils.read_data(file_path)
        print(file_path)
print(len(x))

data/dataset/text_3.txt
data/dataset/text_7.txt
data/dataset/text_2.txt
data/dataset/text_11.txt
data/dataset/text_8.txt
data/dataset/text_6.txt
data/dataset/text_10.txt
data/dataset/text_4.txt
data/dataset/text_9.txt
data/dataset/text_1.txt
data/dataset/text_5.txt
data/dataset/text_13.txt
data/dataset/text_12.txt
93093


In [112]:
aux_data={}
def add(w1,w2):
    key=w1+"_"+w2
    key2=w2+"_"+w1
    if key not in aux_data:
        aux_data[key]=0
        aux_data[key2]=0
    aux_data[key]+=1
    aux_data[key2]+=1
        
for line in x:
    le=len(line)
    for i in range(le):
        w1=line[i]
        if i+1<le:
            w2=line[i+1]
            add(w1,w2)
        if i+2<le:
            w2=line[i+1]
            add(w1,w2)
utils.dump_pickle(dump=aux_data,filename="aux_ds.pkl")

### Lemmatization

In [104]:
# def lemmatize(lines):
#     new_lines=[]
#     lemmatizer=WordNetLemmatizer()
#     counter=0
#     for line in lines:
#         counter+=1
#         if(counter%10000==0):
#             print(counter)
#         new_line=[]
#         for word in line:
#             new_line.append(lemmatizer.lemmatize(word))
#         new_lines.append(new_line)
#     return new_lines

In [94]:
lemm_x=utils.lemmatize(x)
# utils.dump_pickle(lemm_x,"lemm_x.pkl")

10000
20000
30000
40000
50000
60000
70000
80000
90000


## Training Word2Vec

In [105]:
def my_train(x,size,epochs,model,mp="",pret=False):
    model_2 = Word2Vec(size=300, min_count=1,workers=4,sg=1)
    model_2.build_vocab(x)
    total_examples = model_2.corpus_count
    print(total_examples)
    print(len(list(model_2.wv.vocab.keys())))
    print(list(model_2.wv.vocab.keys())[0:10])
    total_examples = model_2.corpus_count
    model_path="sg_emb"+str(size)+"_ep"+str(epochs)+".pkl"
    if pret:
        model_2.intersect_word2vec_format(pretrained_vectors, binary=True, lockf=1.0)
        model_path="pre_"+model_path
    model_path=mp+model_path
    model_2.train(x, total_examples=total_examples, epochs=epochs,report_delay=20)
    utils.dump_pickle(model_2,model_path)

### Loading Google Vector

In [96]:
# model = gensim.models.KeyedVectors.load_word2vec_format(pretrained_vectors, binary=True)  

In [97]:
model

In [106]:
my_train(lemm_x,300,10,model,"lemm_",True)

93093
24271
['the', 'lay', 'man', 's', 'sermon', 'upon', 'late', 'storm', 'held', 'forth']


In [102]:
my_train(lemm_x,300,10,model,"lemm_",False)

93093
24271
['the', 'lay', 'man', 's', 'sermon', 'upon', 'late', 'storm', 'held', 'forth']


In [103]:
my_train(lemm_x,200,10,model,"lemm_",False)

93093
24271
['the', 'lay', 'man', 's', 'sermon', 'upon', 'late', 'storm', 'held', 'forth']


## Fast-text

In [29]:
model2 = FastText(min_count=1,workers=4,min_n=1)
model2.build_vocab(x)

In [30]:
model2.corpus_count,model2.epochs

(93093, 5)

In [31]:
model2.train(x, total_examples=model2.corpus_count, epochs=40)

In [32]:
model_path="fsttxt_emb100_ep40.pkl"
utils.dump_pickle(model2,model_path)

In [87]:
pretrained_vectors

'data/GoogleNews-vectors-negative300.bin.gz'

In [88]:
model